### OvR 모듈 활용


### (1) 모듈 준비 and 데이터 준비
<hr>

In [10]:

from sklearnex import patch_sklearn, unpatch_sklearn
patch_sklearn()

import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
# 나눔고딕
mpl.rc('font', family='NanumGothic')
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.pipeline import Pipeline



Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [11]:
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier


In [12]:
df = pd.read_csv('../data/fish.csv')
df.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


### (2) 학습 데이터셋 준비

#### (2-1) 피쳐/타켓 분리

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Species', axis=1), df['Species'], test_size=0.3, random_state=42)


In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((111, 5), (48, 5), (111,), (48,))

#### (3) 학습 진행
<hr>

#### (3-1) OVO Estimator 생성

In [20]:
## OVO 에서 사용할 관측지(Estimator) 생성
model = LogisticRegression(random_state=42, max_iter=10**10)
# model.fit(X_train, y_train)
## OVO 모델 생성
ovo_model = OneVsOneClassifier(model)
ovo_model.fit(X_train, y_train)



OneVsOneClassifier(estimator=LogisticRegression(max_iter=10000000000,
                                                random_state=42))

In [21]:
attributes = ['classes_', "feature_names_in_", 'estimators_']

for attr in attributes:
    print(f'{attr} : {len(getattr(ovo_model, attr))}')

classes_ : 7
feature_names_in_ : 5
estimators_ : 21


In [22]:
# check train score and test score
print(f'train score : {ovo_model.score(X_train, y_train)}')
print(f'test score : {ovo_model.score(X_test, y_test)}')

train score : 0.9819819819819819
test score : 0.9166666666666666


In [23]:
ovo_model.predict(X_test[:2])

array(['Perch', 'Smelt'], dtype=object)

In [24]:
ovo_model.decision_function(X_test[:2])

array([[ 0.67703491,  4.33136571,  6.32570823, -0.32855841,  5.32469125,
         2.68054385,  1.66929721],
       [ 0.67293299,  3.3316962 ,  5.32447812, -0.32993138,  4.31933802,
         6.32653206,  1.66872554]])

#### (3-2) OVR Estimator 생성

In [25]:
ovr_model = OneVsRestClassifier(LogisticRegression(random_state=42, max_iter=10**10))
ovr_model.fit(X_train, y_train)


OneVsRestClassifier(estimator=LogisticRegression(max_iter=10000000000,
                                                 random_state=42))

In [26]:
attributes = ['classes_', "feature_names_in_", 'estimators_']
for attr in attributes:
    print(f'{attr} : {len(getattr(ovr_model, attr))}')

classes_ : 7
feature_names_in_ : 5
estimators_ : 7


In [29]:
ovr_model.score(X_train, y_train), ovr_model.score(X_test, y_test)

(0.963963963963964, 0.9375)